Nosso projeto consiste em avaliar o processo de visto americano para trabalho, o H1B, e se dependendo de certos fatores uma pessa receberia o visto ou não. Para tirar este visto é necessário passar informações como, o nome do lugar de trabalho, com o que a pessoa trabalhará, se é um trabalho de tempo integral, o salário, e a cidade onde será o trabalho (traduzido para latitude e longitude), todos dados contidos no nosso dataset. Porém havia um problema nosso dataset, obtido no Kaggle havia 3 milhões de linhas, um arquivo muito grande, fazendo com que fique difícil e demorado para se trabalhar. Por esse motivo cortamos o dataset, no qual o processo esta em um outro notebook chamado Shuffle dataset.

In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
df = pd.read_csv('Dataframe.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,0,824764,1,ERNST & YOUNG U.S. LLP,COMPUTER SYSTEMS ANALYSTS,ADVISORY MANAGER,Y,103667.0,2015.0,"HOUSTON, TEXAS",-95.369803,29.760427
1,1,50636,1,CORNING OPTICAL COMMUNICATIONS LLC,MARKET RESEARCH ANALYSTS AND MARKETING SPECIAL...,STRATEGIC PLANNING ANALYST,Y,82139.0,2016.0,"HICKORY, NORTH CAROLINA",-81.344457,35.734454
2,2,1258118,1,SOCIAL MEDIA EDUCATION GROUP PTY LTD,OPERATIONS RESEARCH ANALYSTS,VICE PRESIDENT OF CLIENT SERVICES,Y,64210.0,2015.0,"SAN FRANCISCO, CALIFORNIA",-122.419415,37.774929
3,3,2691274,0,GENPACT PROCESS SOLUTIONS LLC,Accountants,ASSISTANT MANAGER,Y,62546.0,2011.0,"STAMFORD, CONNECTICUT",-73.538734,41.053430
4,4,2680263,0,"NETWORK OBJECTS, INC.",Computer Programmers,PROGRAMMER ANALYST,Y,63586.0,2011.0,"HOFFMAN ESTATES, ILLINOIS",-88.122720,42.062992


Na célula abaixo, o tratamento dos dados é feito. Em primeiro lugar as colunas Unnamed: 0 e Unnamed: 0.1, são retiradas, essas correspondem ao index atual e ao index antes do embaralhamento feito no outro notebook. O local de trabalho, WORKSITE, também é retirado uma vez que esse já está no dataset em forma de Latitude e Longitude, variáves quantitavas. Essas então são transformadas para valores de 0 a 1 para facilitar a forma de se trabalhar, bem como a variável de trabalho em tempo intergral, FULL_TIME_POSITION, também é transformada, de N ou Y, para 0 ou 1. JOB_TITLE é retirado pois a variável SOC_NAME, do Inglês Standard Occupational Classification (SOC), já corresponde ao trabalho ocupado porém em uma forma de classificação padrão e essa então é transormada para uma váriavel quantitativa através do metodo de One-hot Encoding. Por fim o ano é retirado uma vez que queremos prever o visto de anos futuros então não se pode levar em conta os anos passados.  

In [3]:
def dummify(data, column_name):
    df = data.copy()
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name)], axis=1)
    return df2

def mapp (value, start1, stop1, start2, stop2):
    return start2 + (stop2 - start2) * ((value - start1) / (stop1 - start1))


df = df.drop(["WORKSITE","YEAR","JOB_TITLE","Unnamed: 0","Unnamed: 0.1"], axis=1)
df["SOC_NAME"] = df["SOC_NAME"].apply(lambda x: x.upper())

df = df.drop(["EMPLOYER_NAME"], axis=1)

df["FULL_TIME_POSITION"] = df["FULL_TIME_POSITION"].apply(lambda x: 0 if x=="N" else 1)



latmin = df["lat"].min()
latmax = df["lat"].max()
lonmin = df["lon"].min()
lonmax = df["lon"].max()

df["lat"] = df["lat"].apply(lambda x: mapp(x, latmin, latmax, 0, 1))
df["lon"] = df["lon"].apply(lambda x: mapp(x, lonmin, lonmax, 0, 1))

df = dummify(df, "SOC_NAME")

In [4]:
df.head()

,CASE_STATUS,FULL_TIME_POSITION,PREVAILING_WAGE,lon,lat,SOC_NAME_<FONT><FONT>CARPINTEROS</FONT></FONT>,SOC_NAME_ABLE SEAMEN,SOC_NAME_ACCOUNTANTS,SOC_NAME_ACCOUNTANTS AND AUDITORS,SOC_NAME_ACTORS,...,SOC_NAME_VOCATIONAL EDUCATION TEACHERS POSTSECONDARY,"SOC_NAME_VOCATIONAL EDUCATION TEACHERS, MIDDLE SCHOOL","SOC_NAME_VOCATIONAL EDUCATION TEACHERS, POSTSECONDARY",SOC_NAME_WATCH REPAIRERS,SOC_NAME_WEB ADMINISTRATORS,SOC_NAME_WEB DEVELOPERS,"SOC_NAME_WELDERS, CUTTERS, SOLDERERS, AND BRAZERS","SOC_NAME_WHOLESALE AND RETAIL BUYERS, EXCEPT FARM PRODUCTS",SOC_NAME_WRITERS AND AUTHORS,SOC_NAME_ZOOLOGISTS AND WILDLIFE BIOLOGISTS
0,1,1,103667.0,0.205833,0.317569,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,82139.0,0.252032,0.433794,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,64210.0,0.116734,0.473491,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,62546.0,0.277743,0.537275,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,63586.0,0.229705,0.556916,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


O X e y do modelo, baseando-se no CASE_STATUS(se foi aprovado ou não) são separados em base para treinamentos e base para teste, sendo a divisão feita de 80 e 20 por cento. Em seguida é aplicado o modelo, neste caso de regressão logística, uma vez que esse método consegue fazer a predição de para valores a partir das nossas variáves que foram transformadas em quantitativas.

In [5]:
col = []

for i in df.columns:
    if i.startswith("SOC_NAME_"):
        col.append(i)

X = df.drop(["CASE_STATUS"], axis=1).values
y = df["CASE_STATUS"].values

index = round(df.index.size*0.8)

X_train, X_test = X[:index], X[index:]
y_train, y_test = y[:index], y[index:]

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


model = LogisticRegression(solver="lbfgs")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Acurácia: {}".format(100*accuracy_score(y_test, y_pred)))

Acurácia: 50.01


In [7]:
df.head()

,CASE_STATUS,FULL_TIME_POSITION,PREVAILING_WAGE,lon,lat,SOC_NAME_<FONT><FONT>CARPINTEROS</FONT></FONT>,SOC_NAME_ABLE SEAMEN,SOC_NAME_ACCOUNTANTS,SOC_NAME_ACCOUNTANTS AND AUDITORS,SOC_NAME_ACTORS,...,SOC_NAME_VOCATIONAL EDUCATION TEACHERS POSTSECONDARY,"SOC_NAME_VOCATIONAL EDUCATION TEACHERS, MIDDLE SCHOOL","SOC_NAME_VOCATIONAL EDUCATION TEACHERS, POSTSECONDARY",SOC_NAME_WATCH REPAIRERS,SOC_NAME_WEB ADMINISTRATORS,SOC_NAME_WEB DEVELOPERS,"SOC_NAME_WELDERS, CUTTERS, SOLDERERS, AND BRAZERS","SOC_NAME_WHOLESALE AND RETAIL BUYERS, EXCEPT FARM PRODUCTS",SOC_NAME_WRITERS AND AUTHORS,SOC_NAME_ZOOLOGISTS AND WILDLIFE BIOLOGISTS
0,1,1,103667.0,0.205833,0.317569,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,82139.0,0.252032,0.433794,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,64210.0,0.116734,0.473491,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,62546.0,0.277743,0.537275,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,63586.0,0.229705,0.556916,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
